# OpenPose Jupyter Data Exploration
## Imports and dependencies

In [1]:
import pandas as pd
from pandas import DataFrame

import plotly.express as px
import plotly.graph_objects as go
from glob import glob

from path import Path

import os

Mandatory input:
* frame data path
* frame files count
* persons in frame count

## Import data

In [2]:
def get_frame_data(frame_file):
    try:
        frame_data = pd.read_csv(frame_file)
        
        return frame_data
    except FileNotFoundError:
        print(f"Frame data not found for frame {frame_idx}")  
        return pd.DataFrame()

In [3]:
def get_person_files(input_files, frame_data_path):
    all_persons_files = {}
    max_persons = 10
    
    for input_file_idx in range(len(input_files)):
        for person_idx in range(max_persons):
            expected_file = frame_data_path.replace("[frame_idx]", str(input_file_idx))
            expected_file = expected_file.replace("[person_idx]", str(person_idx))
            
            if expected_file in input_files:
                person_files = []
                
                if person_idx in all_persons_files:
                    person_files = all_persons_files[person_idx]
                
                person_files.append(expected_file)
                
                all_persons_files[person_idx] = person_files
                
    persons_indeces = all_persons_files.keys()
    print(f"Found {len(persons_indeces)} persons")
    for person_idx in persons_indeces:
        print(f"Person {person_idx} has {len(all_persons_files[person_idx])} frame files")
    
    return all_persons_files      

In [4]:
def get_frames_data(frames_dir, wrapper_dir):
    frames_root_path = f"/Users/allarviinamae/EduWorkspace/openpose-jupyter-data-exploration/{wrapper_dir}/{frames_dir}"
    frame_data_path = f"{frames_root_path}/{frames_dir}.mov-[frame_idx]-[person_idx].csv"

    print(f"Frame data path={frame_data_path}")

    input_files = [Path(f).abspath() for f in glob(frames_root_path + '/*')]

    input_files_count = len(input_files)
    print(f"Root path includes {input_files_count} files")

    all_persons_files = get_person_files(input_files, frame_data_path)

    # Collecting frame data for the person with index 0
    person_idx_to_collect = 0
    person_frame_files = all_persons_files[person_idx_to_collect]
    
    return [get_frame_data(frame_file) for frame_file in person_frame_files]
 

## Animation

In [5]:
def get_flipped(frame_df_orig, frames_max_x, frames_max_y):
    df = frame_df_orig.copy()
        
    df['y'] *= -1 # flip
    
    #df['y'] += frames_max_y # shift to positive coordinates
    
    return df

def get_body_part(body_connection, frame_df_orig, frames_max_x, frames_max_y):
    frm = body_connection[0]
    to = body_connection[1]
    
    df = get_flipped(frame_df_orig, frames_max_x, frames_max_y)
    
    #if df.iloc[to, 0] == 0 or df.iloc[to, 1] == 0:
    #    return
    #if df.iloc[frm, 0] == 0 or df.iloc[frm, 1] == 0:
    #    return
    
    x0 = df.iloc[frm, 0]
    y0 = df.iloc[frm, 1]
    
    x1 = df.iloc[to, 0]
    y1 = df.iloc[to, 1]
    
    #if body_connection[2] == 'LThigh':
    #    print(f"x0: {x0}, y0: {y0}")
    
    return go.Scatter(x=[x0, x1], y=[y0, y1], name=body_connection[2])

def get_all_body_parts(frame_df, frames_max_x, frames_max_y):
    body_connections = [(0, 1, 'Neck'),
                        (15, 0, 'REye'),
                        (16, 0, 'LEye'),
                        (1, 8, 'Spine'),
                        (1, 2, 'RShoulder'),
                        (2, 3, 'RArm'),
                        (3, 4, 'RForearm'),
                        (1, 5, 'LShoulder'),
                        (5, 6, 'LArm'),
                        (6, 7, 'LForearm'),
                        (8, 9, 'RHip'),
                        (9, 10, 'RThigh'),
                        (10, 11, 'RCalf'),
                        (11, 22, 'RFoot'),
                        (8, 12, 'LHip'),
                        (12, 13, 'LThigh'),
                        (13, 14, 'LCalf'),
                        (14, 19, 'LFoot')]
    
    return [get_body_part(body_connection, frame_df, frames_max_x, frames_max_y) for body_connection in body_connections if get_body_part(body_connection, frame_df, frames_max_x, frames_max_y) != None]  

def get_frames(frame_data, frames_max_x, frames_max_y):
    return [go.Frame(data=get_all_body_parts(frame, frames_max_x, frames_max_y)) for frame in frame_data]

def get_max_y_coord(frame_data):
    return max([max(frame.iloc[:, 1]) for frame in frame_data])

def get_max_x_coord(frame_data):
    return max([max(frame.iloc[:, 0]) for frame in frame_data])

def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

def get_animated_figure(frames_list):
    frames_max_x = get_max_x_coord(frames_list)
    frames_max_y = get_max_y_coord(frames_list)
    
    foo_data = get_all_body_parts(frames_list[0], frames_max_x, frames_max_y)
    print(f"num of body parts: {len(foo_data)}")
    
    fig = go.Figure(
        data = get_all_body_parts(frames_list[0], frames_max_x, frames_max_y),
        layout = go.Layout(
            xaxis=dict(range=[-1000, 1000], autorange=False),
            yaxis=dict(range=[-1000, 1000], autorange=False),
            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None, frame_args(50)]
                             ),
                         dict(label="Pause",
                              method="animate",
                              args=[[None], frame_args(0)]
                             )
                        ]
            )]
        ),
        frames=get_frames(frames_list, frames_max_x, frames_max_y)
    )
    
    return fig

In [ ]:
frames_dirs = [name for name in os.listdir(f"./centered-keypoints") if os.path.isdir(f"./centered-keypoints/{name}")]

for idx, frames_dir in enumerate(frames_dirs):    
    frames_list = get_frames_data(frames_dir, "centered-keypoints")
        
    fig = get_animated_figure(frames_list)
    fig.write_html(f"centered-animations/{frames_dir}.html")

Frame data path=/Users/allarviinamae/EduWorkspace/openpose-jupyter-data-exploration/centered-keypoints/backflip-40-margus/backflip-40-margus.mov-[frame_idx]-[person_idx].csv
Root path includes 83 files
Found 1 persons
Person 0 has 83 frame files
num of body parts: 18
Frame data path=/Users/allarviinamae/EduWorkspace/openpose-jupyter-data-exploration/centered-keypoints/flack-31-rasmus/flack-31-rasmus.mov-[frame_idx]-[person_idx].csv
Root path includes 96 files
Found 1 persons
Person 0 has 96 frame files
num of body parts: 18
Frame data path=/Users/allarviinamae/EduWorkspace/openpose-jupyter-data-exploration/centered-keypoints/flack-19-rasmus/flack-19-rasmus.mov-[frame_idx]-[person_idx].csv
Root path includes 97 files
Found 1 persons
Person 0 has 97 frame files
num of body parts: 18
Frame data path=/Users/allarviinamae/EduWorkspace/openpose-jupyter-data-exploration/centered-keypoints/flack-59-martin/flack-59-martin.mov-[frame_idx]-[person_idx].csv
Root path includes 95 files
Found 1 pers